## 使用Xception backbone做 Trnasfer Learning

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# 我们为了防止碎片化和为了更好的利用内存，tensorflow初始化时会默认占用全部显存https://blog.csdn.net/tsyccnh/article/details/102938368
config = tf.compat.v1.ConfigProto() #allow_soft_placement=True
config.gpu_options.allow_growth = True #dynamically grow the memory used on the GPU
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # 使用90%的显存
#config.log_device_placement = True  # to log device placement (on which device the operation ran)
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                            # input_tensor=input_tensor,pooling=None, classes=2)
'''Resnet 50 架構'''
model=keras.applications.ResNet50(input_shape=(32,32,3), include_top=False, weights='imagenet',pooling=None, classes=10)
model.summary()


Using TensorFlow backend.
D:\Anaconda\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## 添加層數

In [2]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
x=Dropout(rate=0.1)(x)
predictions = Dense(units=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 179


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 93s 2ms/step - loss: 1.2406 - accuracy: 0.5885
Epoch 2/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.9165 - accuracy: 0.6928 0s - loss: 0.9
Epoch 3/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.8767 - accuracy: 0.7022
Epoch 4/100
50000/50000 [==============================] - 81s 2ms/step - loss: 0.7659 - accuracy: 0.7374
Epoch 5/100
50000/50000 [==============================] - 86s 2ms/step - loss: 0.6774 - accuracy: 0.7708
Epoch 6/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.5719 - accuracy: 0.8005
Epoch 7/100
50000/50000 [==============================] - 85s 2ms/step - loss: 0.4703 - accuracy: 0.8369
Epoch 8/100
50000/50000 [==============================] - 86s 2ms/step - loss: 0.4056 - accuracy: 0.8607
Epoch 9/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3414 - accuracy: 0.8819
Epoch 10/100
50000/50000 [=====